# Laboratorio: Recommendation con Algebra Lineare e NumPy

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Test librerie

- NumPy si trova preinstallato su Colab e Anaconda
- Eseguire la seguente cella per verificare il corretto funzionamento di NumPy

In [1]:
import numpy as np
np.array([2, 6, 10, 24]).sum()

42

- In caso di errore, verificare di stare utilizzando il kernel corretto e che NumPy sia installato in esso
- Per installare NumPy all'interno dell'ambiente virtuale corrente (può dare errore per permessi insufficienti in caso non si stia utilizzando un ambiente virtuale):

In [ ]:
# rimuovere il "#" nella riga sotto per abilitare il comando
#pip install numpy

## Recommendation: da strutture dati Python a matrici NumPy

- Nella scorsa esercitazione abbiamo introdotto la **recommendation** di prodotti
  - sappiamo **quali utenti** di un sito di e-commerce **hanno acquistato quali prodotti**
  - vogliamo fornire **suggerimenti personalizzati a ciascun utente** su quali ulteriori prodotti dovrebbe acquistare
- Abbiamo implementato una soluzione utilizzando **strutture dati e funzioni standard di Python**
  - dizionari, insiemi, comprehension, ...
- In questa esercitazione ripetiamo gli stessi passaggi usando **array NumPy**
- Vediamo così come un problema reale si possa modellare nell'ambito dell'algebra lineare e risolvere tramite semplici operazioni tra matrici

### Scaricamento file dati

- Riutilizziamo i dati della scorsa esercitazione, scaricabili all'URL https://git.io/fhxQh
- Eseguire la seguente cella di codice per scaricare il file ZIP dall'URL sopra ed estrarne i file

In [2]:
import os.path
if not os.path.exists("purchases_data.zip"):
    from urllib.request import urlretrieve
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    from zipfile import ZipFile
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

### Caricamento Dati

- Ripetiamo i passaggi della scorsa esercitazione per caricare i dati dai file
  - il file `users.csv` contiene ID e nomi degli utenti analizzati: li carichiamo in un dizionario `users` che associ i nomi agli ID
  - facciamo lo stesso col file `items.csv`, che contiene ID e nomi dei prodotti: otteniamo un dizionario `items`
  - il file `purchases-2000.csv` contiene gli acquisti fino alla fine del 2000 in forma di tuple ID utente + ID prodotto: le carichiamo in un insieme `purchases_set`

In [3]:
import csv
with open("users.csv", "r") as f:
    users = {int(uid): name for uid, name in csv.reader(f, delimiter=";")}
with open("items.csv", "r") as f:
    items = {int(iid): name for iid, name in csv.reader(f, delimiter=";")}
with open("purchases-2000.csv", "r") as f:
    purchases_set = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}

## Algebra Lineare e NumPy

- Oggetto di studio dell'**algebra lineare** sono i vettori, le matrici e le principali operazioni tra di essi
  - con vettori e matrici si possono rappresentare dati da analizzare
  - informazioni d'interesse si possono estrarre con operazioni come il prodotto tra matrici
- **NumPy** è una libreria Python molto usata per lavorare con vettori, matrici e array N-dimensionali in generale
  - sugli array NumPy si possono compiere efficientemente operazioni elemento per elemento, aggregazioni, operazioni di algebra lineare, ecc.
- Iniziando importando il package `numpy` assegnandogli l'alias convenzionale `np`

In [4]:
import numpy as np

## Rappresentare gli acquisti in forma di matrice

- L'insieme `purchases` degli acquisti degli utenti analizzati può essere rappresentato in una matrice binaria (ovvero di valori 0 e 1)
  - ogni **riga** corrisponde ad un **utente**
  - ogni **colonna** corrisponde ad un **prodotto** distinto
  - il valore della cella _i,j_ è 1 se l'utente i ha acquistato il prodotto j, 0 altrimenti

$$\mathbf{P} = \begin{bmatrix}0&1&0&1&0&\cdots\\1&0&0&1&0&\cdots\\0&0&1&0&1&\ldots\\\vdots&\vdots&\vdots&\vdots&\vdots&\ddots\end{bmatrix}$$

- Vediamo come costruire tale matrice

### Assegnazione di indici a utenti e prodotti

- Iniziamo definendo **a quali utenti e prodotti corrispondano** rispettivamente **righe e colonne** della matrice
- Costruiamo un dizionario `user_indices` che associ ad ognuno degli N ID utente (`uid`) un numero tra 0 e N-1 (`index`)
  - usiamo `enumerate` per ottenere tuple `(index, uid)`, dove _index_ è un numero progressivo da 0 a N-1  
    `enumerate([X, Y, Z])` $\Rightarrow$ `(0, X), (1, Y), (2, Z)`
  - usiamo `sorted` per ottenere le chiavi in ordine crescente
    - non è strettamente necessario, ma garantisce la piena riproducibilità dei passaggi

In [22]:
# Prendo tutti gli id caricati e creo un dizionario di tipo:
#    {uid: index, uid: index, ...)
# L'indice lo ottengo semplicemente iterando sulla lista di id
user_indices = {uid: index for index, uid in enumerate(sorted(users.keys()))}

# Con questo modo, invece, il dizionario avrà forma:
#    [(index, uid), (index, uid), ...]
#user_indices = list(enumerate(sorted(users.keys())))
# Così però non funziona il comando: user_indices[uid]

In [23]:
user_indices

{84: 0,
 7661: 1,
 22759: 2,
 27153: 3,
 42298: 4,
 44082: 5,
 48144: 6,
 52797: 7,
 58135: 8,
 62023: 9,
 63776: 10,
 81819: 11,
 82397: 12,
 83632: 13,
 93938: 14,
 109403: 15,
 110576: 16,
 116105: 17,
 116614: 18,
 125462: 19,
 125883: 20,
 126922: 21,
 130054: 22,
 138128: 23,
 140844: 24,
 158428: 25,
 158855: 26,
 164121: 27,
 165618: 28,
 172716: 29,
 187485: 30,
 188455: 31,
 190242: 32,
 192779: 33,
 218221: 34,
 218789: 35,
 224067: 36,
 229993: 37,
 231204: 38,
 239871: 39,
 260996: 40,
 261279: 41,
 274676: 42,
 279452: 43,
 281883: 44,
 293553: 45,
 296466: 46,
 339981: 47,
 360171: 48,
 362142: 49,
 365071: 50,
 365712: 51,
 368839: 52,
 390353: 53,
 391690: 54,
 393881: 55,
 395248: 56,
 399035: 57,
 424169: 58,
 426926: 59,
 429177: 60,
 431084: 61,
 431136: 62,
 439900: 63,
 442793: 64,
 446291: 65,
 452809: 66,
 458094: 67,
 464319: 68,
 483531: 69,
 493152: 70,
 502896: 71,
 503195: 72,
 508200: 73,
 512280: 74,
 516868: 75,
 529431: 76,
 533414: 77,
 536662: 78,
 5

- Eseguiamo un'operazione simile per ottenere un dizionario `item_indices` con la numerazione degli oggetti

In [8]:
item_indices = {iid: index for index, iid in enumerate(sorted(items.keys()))}

- Abbiamo così ottenuto dizionari che mappano ID utenti e prodotti a righe e colonne della matrice
- Ad esempio, la riga corrispondente all'utente con ID 63776 ha indice...

In [24]:
user_indices[63776]

10

### Inizializzazione della matrice

- Per allocare la matrice degli acquisti, definiamone dapprima la _forma_, ovvero il numero di righe e di colonne
- Questi sono pari rispettivamente al numero di utenti e di prodotti, che salviamo in due variabili per comodità

In [18]:
n_users = len(users)
n_items = len(items)
print(f"{n_users} utenti, {n_items} prodotti")

178 utenti, 3384 prodotti


- Creiamo ora la matrice `purchases` inizializzando tutti i valori a 0, ovvero senza alcun acquisto
- Usiamo per questo la funzione `zeros` passando la forma desiderata della matrice
- Dato che la matrice conterrà solo valori interi (0 e 1), specifichiamo `int` come datatype
  - `int` verrà convertito ad un dtype con una precisione specifica, es. `int64`

In [20]:
purchases = np.zeros((n_users, n_items), dtype=int)

### Esercizio 1: Scrittura acquisti nella matrice

- Per ottenere la matrice degli acquisti effettiva, impostare a 1 gli elementi corrispondenti alle coppie utente-oggetto nell'insieme `purchases_set`
  - usare opportunamente i dizionari `user_indices` e `item_indices` creati sopra
  - per impostare un valore nella matrice:  
    `matrice[indice_riga, indice_colonna] = valore`

- Visualizziamo una porzione della matrice ottenuta...

In [35]:
purchases[-2,-2]=0

In [36]:
for (uid, iid) in purchases_set:
  purchases[user_indices[uid], item_indices[iid]] = 1


In [37]:
purchases[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

- Vediamo ad esempio che l'utente della penultima riga ha acquistato i prodotti di indice 11 e 12

### Array booleani

- Oltre agli array numerici, NumPy supporta array booleani, contenenti valori `False` e `True`
- Abbiamo rappresentato `purchases` con valori 0 e 1 perché è una notazione comune, ma può essere rappresentata anche come array booleano
- Usiamo il metodo `astype` per ottenere una copia della matrice convertendone il datatype
  - viene seguita la convenzione di Python per cui 0 diventa `False` e ogni altro numero `True`

In [38]:
purchases_bool = purchases.astype(bool)

- Otteniamo una matrice con 0 e 1 convertiti rispettivamente in `False` e `True`

In [39]:
purchases_bool[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True,  True, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False]])

- Questa versione booleana della matrice ci servirà per alcune operazioni

## Associare nomi a righe e colonne

- I dizionari `*_indices` permettono, dato l'ID di un utente o prodotto, di risalire alla riga o colonna corrispondente della matrice
- Creiamo ora un vettore `user_names` che contenga i nomi degli utenti nelle posizioni corrispondenti alle righe della matrice
- Allochiamo un vettore vuoto con lunghezza pari al numero di utenti e tipo di elementi `object`, ovvero oggetti Python arbitrari
  - `n_users` (valore singolo) equivale a `(n_users, )` (tupla di un elemento, quindi vettore)

In [40]:
user_names = np.empty(n_users, dtype=object)

### Esercizio 2: Creazione vettori nomi

- **(2a)** Riempire il vettore `user_names` con i nomi degli utenti corrispondenti alle righe di `purchases`
  - usare i dizionari `users` e `user_indices`
- **(2b)** Creare e riempire in modo simile un vettore `item_names` con i nomi degli oggetti corrispondenti alle colonne di `purchases`

In [51]:
# for uid, index in user_indices.items(): # senza .items() non funziona
#   user_names[index] = (uid, users[uid])

In [56]:
# SOLUZIONE
for uid, name in users.items():
  index = user_indices[uid]
  user_names[index] = name

item_names = np.empty(n_items, dtype = object)
for iid, name in items.items():
  index = item_indices[iid]
  item_names[index] = name

In [55]:
user_names

array(['malachix', 'Kevin Alphonso', 'tropic_of_criticism',
       'Joe Owen "Joe"', 'Zane', 'David Robson', 'Alan R. Holyoak',
       'Mr N Forbes-warren "author of RESURGENCE and ...',
       'Jacques COULARDEAU "A soul doctor, so to say"',
       'Martin Turner "book reviewer"', 'brent been',
       'dsrussell "greyhater"', 'juno2', 'MOVIE_NUT74@HOTMAIL.COM',
       'laddie5', 'Studebaker Hoch, billythemtn@geocities.com',
       'LUCIEN LESSARD', 'Michael J. Berquist',
       'behet@stud.uni-frankfurt.de', 'Alex Udvary', 'flickjunkie',
       'Sean Ares Hirsch', 'lecorel@hotmail.com', 'zara_azari',
       'yygsgsdrassil "yygsgsdrassil"', 'Vincent Tesi "Vinny"',
       'NEO-CS- "Cyber Soldier"', 'Dustin J. Hetke', 'Scott T. Rivers',
       'Aaron Amos', 'jakelamotta', 'A. Andersen', 'snlkidsinhall',
       'S. Schell', 'Dennis Littrell', 'N. Schoenfeld', 'R. Penola',
       'Hayley', 'Seifer', 'Mark Twain "Sam"', 'Sam Damon Jr.',
       'Dennis A. Amith (kndy)', 'carolyn5000', 'scots

- Possiamo così ottenere il nomi di un singolo utente o prodotto data la riga o colonna

In [57]:
item_names[11]

'Doctor Who - Robot [VHS]'

- In più, rispetto alle liste di Python, possiamo estrarre molteplici valori dal vettore passando una **lista di indici**

In [ ]:
item_names[[3112, 1417, 3109]]

array(['Star Wars - Episode IV, A New Hope [VHS]',
       'Star Wars - Episode V, The Empire Strikes Back [VHS]',
       'Star Wars - Episode VI, Return of the Jedi [VHS]'], dtype=object)

## Selezione elementi tramite array booleani

- Possiamo usare l'**indicizzazione booleana** per estrarre elementi che corrispondano ad una condizione
- Consideriamo ad esempio la riga 176 della matrice, che indica gli acquisti dell'utente...

In [ ]:
user_names[176]

'G.Spider'

- Estraiamo la riga corrispondente dalla matrice `purchases_bool` per ottenere un vettore in forma booleana
  - `[176, :]` si può abbreviare in `[176]`
  - essendo una singola riga di una matrice, otteniamo un vettore

In [ ]:
purchased_by_user_176 = purchases_bool[176, :]

# visualizzo i primi 20 valori
purchased_by_user_176[:20]

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False])

- Possiamo usare questo vettore come selettore del vettore `item_names` per estrarre i soli valori di `item_names` corrispondenti a valori `True`
- In questo modo otteniamo un vettore con i **nomi** dei soli prodotti **acquistati dall'utente**

In [ ]:
item_names_purchased_by_user_176 = item_names[purchased_by_user_176]

# visualizzo i primi 10 elementi
item_names_purchased_by_user_176[:10]

array(['Doctor Who - Robot [VHS]', 'Doctor Who - City of Death [VHS]',
       'Doctor Who: The Curse of Peladon [VHS]',
       'Doctor Who:  Time and The Rani [VHS]',
       'Doctor Who - Ghost Light [VHS]',
       'Doctor Who - The Visitation / Black Orchid [VHS]',
       'Doctor Who - Kinda [VHS]',
       'Doctor Who - The Mark of the Rani [VHS]',
       'Doctor Who - The Leisure Hive [VHS]',
       'Star Wars - Episode I, The Phantom Menace [VHS]'], dtype=object)

- Si noti che per questa selezione è necessario usare l'array booleano, usando quello binario si ha un risultato errato

In [ ]:
item_names[purchases[176, :]]

array(['Age of Innocence [VHS]', 'Age of Innocence [VHS]',
       'Age of Innocence [VHS]', ..., 'Age of Innocence [VHS]',
       'Age of Innocence [VHS]', 'Age of Innocence [VHS]'], dtype=object)

## Statistiche sugli acquisti

- Gli array forniscono metodi per calcolare valori aggregati: somma, media, min/max, ecc.
- Ad esempio il metodo `sum` calcola la somma di tutti gli elementi di un array
- Possiamo utilizzarlo per ottenere il numero totale di acquisti analizzati

In [ ]:
purchases.sum()

9683

- Posso ottenere lo stesso risultato effettuando la somma sulla matrice booleana (perché trattati come numeri `False` e `True` valgono 0 e 1 rispettivamente)
  - in generale, `sum` può essere usato per contare i valori `True` in un array booleano

In [ ]:
purchases_bool.sum()

9683

- Oltre ad aggregare tutti i valori, possiamo compiere aggregazioni per righe (asse 0) o per colonne (asse 1)
- Applichiamo la somma per righe e per colonne alla matrice degli acquisti
  - `sum(1)` -> sommo tra loro le colonne -> ottengo il numero di prodotti acquistati per ogni utente
  - `sum(0)` -> sommo tra loro le righe -> ottengo il numero di utenti che hanno acquistato ciascun prodotto

In [65]:
user_purchases = purchases.sum(1) # Sommo lungo le colonne (utenti)
item_purchases = purchases.sum(0) # Sommo lungo le righe (oggetti)

In [ ]:
# numero di acquisti dei primi 10 clienti
user_purchases[:10]

array([38, 32, 50, 41, 34, 47, 72, 40, 33, 51])

In [ ]:
# numero di clienti che hanno acquistato i primi 10 oggetti
item_purchases[:10]

array([1, 2, 1, 4, 1, 1, 1, 1, 1, 2])

Oltre a `sum` si ricordano tra i metodi più comuni
- `mean` per la media
- `min` e `max` per ottenere **i valori** minimo e massimo dell'array
- `argmin` e `argmax` per ottenere **gli indici** all'interno di un vettore del valori minimo e massimo

Nota: per ognuno di questi metodi, ad es. `X.sum()`, esiste una funzione equivalente `np.sum(X)` che si può applicare anche ad oggetti "array-like", ad es. le normali `list` di Python

In [58]:
np.sum( np.array([1, 2, 3]) )

6

In [ ]:
np.sum([1, 2, 3])

6

### Esercizio 3: Estrazione informazioni dai vettori

Utilizzando i vettori `user_purchases`, `item_purchases`, `user_names` e `item_names` ricavare:
- **(3a)** il numero di acquisti medio per ogni utente
- **(3b)** il numero di acquisti del prodotto più acquistato
- **(3c)** il nome dell'utente che ha effettuato più acquisti
- **(3d)** il numero di utenti che hanno acquistato almeno 50 prodotti
- **(3e)** i nomi dei prodotti acquistati da almeno 35 utenti

In [59]:
# 3a
purchases.sum(1).mean()

54.39887640449438

In [60]:
# 3b
purchases.sum(0).max()

50

In [68]:
# 3c
# purchases.sum(1).max()

# SOLUZIONE
user_names[user_purchases.argmax()]

'Bertin Ramirez "justareviewer"'

In [72]:
# len(list(filter(lambda x: x >= 50, purchases.sum(1))))

# SOLUZIONE
(user_purchases >= 50).sum()

68

In [77]:
# SOLUZIONE
# Cosa mi interessa? un nome da item_names ==>   item_names[]
# Come filtro i dati? ==> uso semplicemente l'espressione di filtro
item_names[item_purchases >= 35]

array(['Star Wars - Episode I, The Phantom Menace [VHS]', 'The Matrix',
       'Sleepy Hollow', 'Curse of the Blair Witch [VHS]',
       'The Sixth Sense [VHS]'], dtype=object)

## Similarità tra utenti

- Misuriamo la similarità tra due utenti come il numero di **prodotti distinti acquistati da entrambi**
- Rappresentando i dati in insiemi Python, abbiamo usato intersezioni tra gli insiemi di prodotti acquistati
- Avendo un vettore (riga della matrice) booleano per ogni utente, possiamo trovare quali sono gli acquisti comuni tra due utenti tramite l'AND elemento per elemento
- Estraiamo ad esempio gli acquisti comuni tra i primi due utenti nella matrice

In [ ]:
common_purchases_0_1 = purchases_bool[0] & purchases_bool[1]

In [ ]:
common_purchases_0_1

array([False, False, False, ..., False, False, False])

- Nella rappresentazione binaria (0 e 1), questo corrisponde alla moltiplicazione elemento per elemento
  - il prodotto elemento per elemento è 1 solo dove in entrambi i vettori c'è 1, altrimenti è 0

In [ ]:
common_purchases_0_1 = purchases[0] * purchases[1]

In [ ]:
common_purchases_0_1

array([0, 0, 0, ..., 0, 0, 0])

- Per sapere quanti sono i prodotti acquistati in comune posso contare i `True` o gli 1 nell'array col metodo `sum`

In [ ]:
common_purchases_0_1.sum()

4

- Il numero di prodotti acquistati in comune è quindi pari alla somma dei prodotti tra le corrispondenti righe della matrice

In [ ]:
np.sum( purchases[0] * purchases[1] )

4

- Questo corrisponde al **prodotto scalare** (_dot product_) tra le due righe, che può essere estratto efficientemente con la funzione `dot`, il metodo `dot` o l'operatore `@`

In [ ]:
np.dot(purchases[0], purchases[1])

4

In [ ]:
purchases[0].dot(purchases[1])

4

In [ ]:
purchases[0] @ purchases[1]

4

### Matrice di similarità

- Vogliamo ottenere in una matrice `similarity` NxN tutte le similarità reciproche tra gli N utenti, ovvero i prodotti scalari tra tutte le righe di `purchases`

```
similarity[i, j] == purchases[i, :] @ purchases[j, :]
```

- Abbiamo visto che il **prodotto tra matrici** restituisce una matrice di prodotti scalari tra le righe di una matrice e le colonne di un'altra

```
(A @ B)[i, j] == A[i, :] @ B[:, j]
```

- Se vogliamo calcolare i prodotti tra le righe di A e le righe di B, possiamo calcolare il prodotto tra A e la **trasposta** di B, estraibile con `B.T`

```
(A @ B.T)[i, j] == A[i, :] @ B[j, :]
```

- Per ottenere i prodotti scalari tra tutte le righe di `purchases` possiamo quindi **moltiplicarla per la sua trasposta**

In [78]:
similarity = purchases @ purchases.T

- Otteniamo una matrice quadrata, di ordine pari al numero di utenti

In [ ]:
similarity.shape

(178, 178)

- Visualizziamo un estratto della matrice...

In [ ]:
similarity[:10, :10]

array([[38,  4,  1,  0,  1,  2,  1,  1,  0,  2],
       [ 4, 32,  1,  0,  0,  2,  0,  1,  2,  4],
       [ 1,  1, 50,  1,  4,  2,  1,  2,  1,  7],
       [ 0,  0,  1, 41,  1,  1,  3,  1,  1,  2],
       [ 1,  0,  4,  1, 34,  1,  0,  2,  1,  2],
       [ 2,  2,  2,  1,  1, 47,  1,  0,  2,  1],
       [ 1,  0,  1,  3,  0,  1, 72,  1,  1,  2],
       [ 1,  1,  2,  1,  2,  0,  1, 40,  1,  6],
       [ 0,  2,  1,  1,  1,  2,  1,  1, 33,  3],
       [ 2,  4,  7,  2,  2,  1,  2,  6,  3, 51]])

- ...possiamo ad esempio notare che
  - tra i primi due utenti ci sono 4 oggetti acquistati in comune (come visto sopra)
  - il primo ed il quarto utente non hanno alcun acquisto in comune

In [ ]:
similarity[0, 1]

4

In [ ]:
similarity[0, 3]

0

- Possiamo verificare che la matrice sia simmetrica controllando se è uguale alla sua trasposta
  - l'operatore `==` applicato tra matrici restituisce una matrice booleana con il confronto elemento per elemento
  - usiamo il metodo `all` (o la funzione `np.all`) per verificare che tutti i valori siano `True`

In [ ]:
(similarity == similarity.T).all()

True

In [ ]:
np.all(similarity == similarity.T)

True

- In alternativa NumPy offre una funzione `array_equal` per verificare l'uguaglianza tra due array
  - questa evita errori in caso si comparino array di dimensioni incompatibili

In [ ]:
np.array_equal(similarity, similarity.T)

True

### Diagonale della matrice

- La diagonale della matrice creata contiene i prodotti scalari di ciascuna riga della matrice originale con se stessa
- Si può verificare che sono uguali al numero di acquisti per colonne calcolato sopra

In [ ]:
np.array_equal(np.diag(similarity), user_purchases)

True

- Prima di procedere, **impostiamo a 0 tutti i valori sulla diagonale** per far sì che tra i simili di ciascun utente non sia incluso lui stesso
  - la funzione `fill_diagonal` imposta gli elementi della diagonale al valore dato

In [79]:
np.fill_diagonal(similarity, 0)
similarity[:5, :5]

array([[0, 4, 1, 0, 1],
       [4, 0, 1, 0, 0],
       [1, 1, 0, 1, 4],
       [0, 0, 1, 0, 1],
       [1, 0, 4, 1, 0]])

### Esercizio 4: Estrazione informazioni dalla matrice similarità

- **(4a)** Qual è il numero massimo di prodotti in comune tra due utenti?
- **(4b)** Qual è il nome dell'utente che ha più acquisti in comune con l'utente con ID 7661?

In [85]:
similarity.max()

53

In [90]:
user_names[similarity[user_indices[7661]].argmax()]

'brent been'

## Stimare il potenziale interesse nei prodotti

- Per stimare **quanto ciascun utente U sia potenzialmente interessato** in ciascun prodotto P, calcoliamo la somma delle similarità degli altri utenti che l'hanno acquistato
- Con Python abbiamo preso i punteggi di similarità tra U e altri utenti, filtrato quelli dei soli acquirenti di P e calcolato la loro somma
- Questa somma equivale in pratica al prodotto scalare tra:
  - la riga relativa a U di `similarity` che indica i punteggi di similarità con gli altri utenti e
  - la colonna relativa a P di `purchases` che indica quali utenti lo hanno acquistato
- Ad esempio, l'interesse stimato del 1° utente verso il 2° prodotto è:

In [91]:
similarity[0, :] @ purchases[:, 1]

5

- Per ottenere l'interesse di ogni utente U verso ogni prodotto P possiamo quindi usare come sopra il prodotto tra le due matrici
  - la matrice `common_purchases` ha forma **utenti** x utenti
  - la matrice `purchases` ha forma utenti x **prodotti**
  - la matrice risultante avrà forma **utenti x prodotti**

In [92]:
interest = similarity @ purchases

- Verifichiamo che la forma della matrice sia pari a quella della matrice degli acquisti (utenti x prodotti)

In [93]:
interest.shape == purchases.shape

True

- Visualizziamo un estratto della matrice...

In [94]:
interest[0:10, 0:10]

array([[ 0,  5,  0,  4,  4,  3,  1,  0,  4,  3],
       [ 3,  7,  1,  6,  1,  1,  2,  1,  2,  0],
       [ 6,  9,  0, 12,  0,  0,  1,  0,  9,  0],
       [ 1,  2,  2,  8,  0,  0,  0,  2,  5,  2],
       [ 3,  2,  0,  5,  0,  0,  0,  0,  2,  0],
       [ 2,  5,  0, 10,  0,  0,  1,  0,  5,  0],
       [ 0,  5,  5,  5,  1,  2,  1,  5,  2,  3],
       [ 1,  5,  0,  7,  0,  0,  0,  0,  0,  3],
       [ 2,  4,  0,  9,  0,  0,  1,  0,  4,  4],
       [ 6, 17,  0, 25,  0,  0,  0,  0,  9,  2]])

- Ad esempio si ritiene che il 10° utente (ultima riga) sia molto interessato al 4° prodotto

### Esercizio 5: Scartare i prodotti già acquistati

- Nella matrice calcolata `interest`, sono ritenuti "interessanti" anche i prodotti **già acquistati** da ciascun utente
- Prima di proseguire, **impostare a 0** i valori di `interest` relativi a coppie utente-prodotto a cui **corrisponde un acquisto effettuato**

In [96]:
interest[purchases > 0] = 0

In [97]:
interest

array([[ 0,  5,  0, ...,  5,  8, 10],
       [ 3,  7,  1, ...,  7,  5,  6],
       [ 6,  9,  0, ...,  9,  9,  5],
       ...,
       [ 2,  8,  5, ...,  8,  0, 11],
       [ 0,  1,  0, ...,  1,  1,  0],
       [ 3,  6,  0, ...,  6,  8,  5]])

## Ottenere _N_ suggerimenti di acquisto per ogni utente

- Come la scorsa volta, vogliamo suggerire un numero fisso N di prodotti ad ogni utente, scegliendo quelli con interesse previsto maggiore

In [100]:
N = 20

- Per prima cosa assegniamo a ciascun prodotto un "ranking" in base all'interesse per ciascun utente
- Quindi selezioniamo per ogni cliente gli _N_ prodotti col ranking migliore

### Estrarre l'ordine dei valori: il metodo `argsort`

- Il metodo `argsort` su un vettore restituisce un vettore con i suoi **indici** (da 0 a N-1) **ordinati** secondo i valori

In [101]:
x = np.array([320, 80, 20, 40, 160, 640, 10])
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

- Significa che l'elemento minimo è quello di indice 6 (10), seguito da quello di indice 2 (20) e così via fino al massimo di indice 5 (640)
- In pratica, selezionando gli elementi dell'array nell'ordine dato da `argsort`, otteniamo un array con gli elementi in ordine crescente

In [102]:
x[x.argsort()]

array([ 10,  20,  40,  80, 160, 320, 640])

- Nel caso di una matrice, l'operazione va eseguita **lungo una dimensione** a scelta (riga per riga o colonna per colonna)

In [103]:
np.array([[32,  8,  2,  4, 16, 64,  1],
          [ 8, 16,  4, 64, 32,  1,  2]]).argsort(1)   # 1 = riga per riga

array([[6, 2, 3, 1, 4, 0, 5],
       [5, 6, 2, 0, 1, 4, 3]])

### Calcolare il "ranking" dei prodotti

- Applicando l'operazione `argsort` due volte otteniamo il "ranking" dei valore, ovvero l'indice che ogni elemento avrebbe nell'array ordinato
  - otteniamo quindi un array che associa **0 all'elemento più basso, 1 al secondo e così via**

In [104]:
x

array([320,  80,  20,  40, 160, 640,  10])

In [105]:
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

Da leggersi: l'elemento più basso di `x` ha indice 6, il secondo ha indice 2, ...

In [106]:
x.argsort().argsort()

array([5, 3, 1, 2, 4, 6, 0])

Da leggersi: il primo elemento di `x` è il 5° (contando da 0°) più basso, il secondo è il 3°, ...

_(chi fosse curioso può leggere [questa dimostrazione formale](https://www.berkayantmen.com/rank.html) che spiega perché si ottiene questo risultato)_

### Ottenere il ranking dei prodotti per ogni utente

- Tramite l'applicazione doppia di `argsort`, possiamo trasformare i punteggi di `interest` in valori 0, 1, 2, ... che indichino in ciascuna riga il valore più basso, il secondo ecc.
- D'altra parte, **invertendo** l'array a cui è applicata l'operazione, otteniamo un array che associa 0 al valore **più alto**, 1 al secondo ecc.
- Applichiamo queste operazioni per ottenere un array `interest_ranking`

In [107]:
interest_ranking = (-interest).argsort(1).argsort(1) # riga per riga

In [108]:
interest_ranking[:5, :10]

array([[3383, 1259, 2771, 1347, 1346, 1648, 2684, 2775, 1352, 1655],
       [1585,  853, 2324, 1097, 2297, 2300, 1860, 2310, 1854, 2794],
       [1214,  930, 2777,  595, 2874, 2857, 2224, 2773,  925, 2782],
       [2148, 1519, 1515,  359, 2821, 2817, 2816, 1507,  808, 1511],
       [1197, 1739, 2676,  775, 2675, 2672, 2667, 2666, 1750, 2654]])

- La matrice indica ad es. che, per il primo utente, il secondo prodotto è il 1.259° (partendo da 0°) più interessante da acquistare

### Selezionare i prodotti da suggerire

- Abbiamo così ottenuto un array dove per ogni utente (riga) abbiamo i prodotti numerati univocamente da 0 a _N_-1 in ordine di interesse
- Da questo array possiamo quindi selezionare i **valori minori di _N_** per ottenere gli **_N_ prodotti di maggiore interesse** per ciascun utente
  - otteniamo una matrice booleana, che convertiamo in numerica con `astype`

In [109]:
suggestions = (interest_ranking < N).astype(int)

In [111]:
suggestions.sum(1)

array([20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20])

- Questa è la **matrice dei prodotti suggeriti**, che associa ad ogni utente gli _N_ prodotti a cui è potenzialmente più interessato

## Accuratezza dei suggerimenti di acquisto

- Come la scorsa volta, valutiamo i suggerimenti ottenuti comparandoli con gli acquisti effettuati dopo l'analisi
- Tali acquisti sono riportati nel file `purchases-2014.csv`
- Come abbiamo fatto per il primo file, creiamo una matrice binaria che indichi quali utenti hanno acquistato quali prodotti
  - `zeros_like` crea una matrice di zeri di tipo e forma identiche a una data (esistono anche `empty_like` e `ones_like`)

In [139]:
purchases_updated = np.zeros_like(purchases) #crea matrice stessa forma purchases ma tutti zeri

### Esercizio 6: Caricamento dati in matrice

- Impostare a 1 gli elementi di `purchases_updated` corrispondenti alle coppie utente-prodotto nel file CSV `purchases-2014.csv`
  - non usare una struttura dati intermedia (es. l'insieme `purchases_set` usato sopra)
  - riutilizzare gli stessi dizionari `user_indices` e `item_indices` di prima per garantire che righe e colonne delle matrici combacino

In [147]:
with open("purchases-2014.csv", "r") as f:
    # purchases_set = {(int(uid), int(iid)) for uid, iid in csv.reader(f, delimiter=";")}
    for uid, iid in csv.reader(f, delimiter=";"):
      # ATTENZIONE! csv.reader restituisce sempre delle stringhe! Ricorda di convertire ad interi
      purchases_updated[user_indices[int(uid)], item_indices[int(iid)]] = 1

In [148]:
purchases_updated

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Esercizio 7: Selezionare solo i nuovi acquisti

- La nuova matrice riporta **tutti** gli acquisti, compresi quelli già indicati nella matrice precedente
- Vogliamo una matrice binaria `new_purchases` in cui siano indicati **solo gli acquisti successivi** all'analisi svolta sopra
- Quale operazione tra le matrici `purchases` e `purchases_updated` ci permette di ottenere tale risultato?

In [149]:
new_purchases = purchases_updated - purchases

### Esercizio 8: Quali nuovi acquisti sono stati suggeriti?

- Abbiamo ora la matrice `suggestions` con gli acquisti _suggeriti_ e quella `new_purchases` con i nuovi acquisti _effettivi_
- Da queste possiamo estrarre una matrice binaria `hits` che indichi quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto
- Quale operazione tra le due matrici ci fa ottenere tale risultato?

In [150]:
hits = suggestions * new_purchases

In [151]:
hits.sum()

149

### Esercizio 9: Quanti utenti hanno ricevuto almeno un suggerimento valido?

- **(9a)** Ottenere un vettore binario `satisfied_users` che associ 1 ai soli utenti che hanno ottenuto almeno un suggerimento valido, cioè almeno un 1 nella riga corrispondente di `hits`
- **(9b)** Ricavare dal vettore stesso il numero di tali utenti
- **(9c)** Ricavare dal vettore stesso la percentuale di tali utenti rispetto al numero totale

In [152]:
satisfied_users = hits.max(1)

In [154]:
satisfied_users.sum()

61

In [153]:
satisfied_users.mean()

0.34269662921348315

- Si dovrebbe ottenere una percentuale del **34,3%** di utenti soddisfatti, alla pari della scorsa esercitazione

## Esercizio extra: Confronto con una selezione casuale di prodotti

- Come nella scorsa esercitazione, per valutare quanto sia buono il risultato ottenuto, confrontiamolo con quello che si otterrebbe suggerendo prodotti a caso
- Utilizziamo la generazione casuale di array fornita da NumPy, inizializzando il relativo seed

In [155]:
np.random.seed(123)

- **(a)** generare una matrice `random_interest` di forma pari ad `interest` con valori casuali
  - usare una funzione per generare valori casuali in un intervallo continuo, ad es `np.random.random`, indicando la forma desiderata

In [160]:
random_interest = np.zeros_like(interest)

In [161]:
random_interest

<function RandomState.random>

- **(b)** impostare a 0 i valori della matrice corrispondenti a prodotti già acquistati

- **(c)** selezionare gli _N_ prodotti di maggiore interesse per ogni utente
  - come sopra, generare la matrice con i ranking lungo ciascuna riga, quindi selezionare i ranking appropriati

In [157]:
random_interest_ranking = ...
random_suggestions = ...

- **(d)** eseguire il confronto con i prodotti effettivamente acquistati ed estrarre il vettore che indica gli utenti con almeno un suggerimento valido

In [158]:
random_hits = ...
randomly_satisfied_users = ...

- **(e)** calcolare la percentuale di tali utenti rispetto al totale